# Watson OpenScale Mortgage Default Lab

Part one of a hands-on lab for IBM Watson OpenScale, this notebook should be run in a [Watson Studio](https://dataplatform.ibm.com/) project with Python 3.6 or greater. It requires a free lite version of [Watson Machine Learning](https://cloud.ibm.com/catalog/services/machine-learning).

This notebook will train, save and deploy a machine learning model to predict mortgage defaults.

## Provision services and create credentials

You will need credentials for Watson Machine Learning. If you already have a WML instance, you may use credentials for it. To provision a new Lite instance of WML, use the [Cloud catalog](https://cloud.ibm.com/catalog/services/machine-learning), give your service a name, and click **Create**. Once your instance is created, click the **Service Credentials** link on the left side of the screen. Click the **New credential** button, give your credentials a name, and click **Add**. Your new credentials can be accessed by clicking the **View credentials** button. Copy and paste your WML credentials into the cell below.

In [ ]:
WML_CREDENTIALS = {
  "apikey": "xxxxxxxxxxx",
  "iam_apikey_description": "Auto-generated for key xxxxxxx",
  "iam_apikey_name": "xxxxxxx",
  "iam_role_crn": "crn:v1:bluemix:public:iam::::serviceRole:Writer",
  "iam_serviceid_crn": "crn:v1:bluemix:public:iam-identity::xxxxxxx",
  "instance_id": "xxxxxxx",
  "url": "https://us-south.ml.cloud.ibm.com"
}

## Name your model

You may give your model and deployment a custom name below; however, if you change the values below, be sure to use the same names in all subsequent notebooks in this lab.

In [ ]:
MODEL_NAME = 'Mortgage Default'
DEPLOYMENT_NAME = 'Mortgage Default - Production'

## Run the notebook

At this point, you can run all cells in this notebook using the menus above.

Import the scikit-learn framework and check the version. This notebook was developed using sklearn version 0.20.3.

In [ ]:
import sklearn
sklearn.__version__

Use the provided credentials above to create a new Watson Machine Learning client.

In [ ]:
from watson_machine_learning_client import WatsonMachineLearningAPIClient

client = WatsonMachineLearningAPIClient(WML_CREDENTIALS)

List all models for this instance of Watson Machine Learning.

In [ ]:
client.repository.list_models()

Import the pandas library, download and examine our training data. The data contains an 'ID' field for the loan ID, which will not be used in training the model and is dropped.

In [ ]:
import pandas as pd

url = 'https://raw.githubusercontent.com/emartensibm/mortgage-default/master/Mortgage_Full_Records.csv'
df_raw = pd.read_csv(url)
df = df_raw.drop('ID', axis=1)
df.head()

Import the sklearn libraries we need, including encoders, transformers, scalers, and our random forest classifier.

In [ ]:
from sklearn.preprocessing import OneHotEncoder
from sklearn.ensemble import RandomForestClassifier
from sklearn.impute import SimpleImputer
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.metrics import classification_report
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler

Identify the categorical features, and create a one-hot encoder pipeline for them.

Next, identify the numerical features and use the min-max scaler to scale the values, which will significantly increase our model's accuracy.

Finally, organize the categorical encoder and the scaler into a pipeline so the deployed model can work with our data.

In [ ]:
categorical_features = ['AppliedOnline','Residence','Location']
categorical_transformer = Pipeline(steps=[
    ('imputer', SimpleImputer(strategy='constant', fill_value='missing')),
    ('onehot', OneHotEncoder(handle_unknown='ignore'))])

scaled_features = ['Income','Yrs_at_Current_Address','Yrs_with_Current_Employer',\
                   'Number_of_Cards','Creditcard_Debt','Loan_Amount','SalePrice']
scale_transformer = Pipeline(steps=[('scale', MinMaxScaler())])

preprocessor = ColumnTransformer(
    transformers=[
        ('cat', categorical_transformer, categorical_features),
        ('scaler', scale_transformer, scaled_features)
    ]
)

clf = Pipeline(steps=[('preprocessor', preprocessor),
                      ('classifier', RandomForestClassifier())])

Perform the train/test split, train the model, and score the model quality.

In [ ]:
X = df.drop('MortgageDefault', axis=1)
y = df['MortgageDefault']

X_train, X_test, y_train, y_test = train_test_split( X, y, test_size=0.2, random_state=4)

model = clf.fit(X_train, y_train)
res_predict = model.predict(X_test)
print("model score: %.3f" % clf.score(X_test, y_test))
print(classification_report(y_test, res_predict, target_names=["False", "True"]))

## Save the model to Watson Machine Learning

Check the list of models in the WML instance, and remove pre-existing versions of this model. This allows the notebook to be re-run to reset all data if necessary.

In [ ]:
model_deployment_ids = client.deployments.get_uids()
for deployment_id in model_deployment_ids:
    deployment = client.deployments.get_details(deployment_id)
    model_id = deployment['entity']['deployable_asset']['guid']
    if deployment['entity']['name'] == DEPLOYMENT_NAME:
        print('Deleting deployment id', deployment_id)
        client.deployments.delete(deployment_id)
        print('Deleting model id', model_id)
        client.repository.delete(model_id)
client.repository.list_models()

Create the metadata and save the model.

In [ ]:
metadata = {
    client.repository.ModelMetaNames.NAME: MODEL_NAME,
    client.repository.ModelMetaNames.EVALUATION_METHOD: "binary",
    client.repository.ModelMetaNames.EVALUATION_METRICS: [
        {
            "name": "areaUnderROC",
            "value": 0.7,
            "threshold": 0.7
        }
    ]
}

# Name the columns
cols=["Income","AppliedOnline","Residence","Yrs_at_Current_Address","Yrs_with_Current_Employer",\
      "Number_of_Cards","Creditcard_Debt","Loans","Loan_Amount","SalePrice","Location"]
      
saved_model = client.repository.store_model(model=model, meta_props=metadata, 
                                            training_data=X_train, training_target=y_train, 
                                            feature_names=cols, label_column_names=["MortgageDefault"] )
saved_model

Get the unique ID for the model so we can deploy it.

In [ ]:
model_uid = saved_model['metadata']['guid']
model_uid

Deploy the model as a web service with Watson Machine Learning.

In [ ]:
print("Deploying model...")

deployment = client.deployments.create(artifact_uid=model_uid, name=DEPLOYMENT_NAME, asynchronous=False)

In [ ]:
deployment_uid = client.deployments.get_uid(deployment)

print("Model id: {}".format(model_uid))
print("Deployment id: {}".format(deployment_uid))

## Congratulations!

If all cells have run successfully, you have successfully deployed the mortgage default model as a web service in Watson Machine Learning. You can proceed with the rest of the lab.